In [1]:
import pandas as pd
import numpy as np
import os
import torch

In [2]:
root = os.path.join('../data_train')
cat_folders = os.listdir(root)
cat_folders

['MAESTROS DEL HOGAR',
 'SIEMPRE LIMPIOS',
 'BELLEZA Y CUIDADO FAMILIAR',
 'LA BOTI DE CHILE',
 'DULCE MOMENTO',
 'CONGELADOS',
 'PROTEINAS',
 'INDESPENSABLES Y MASCOTAS',
 'HOGAR  TEXTIL HOGAR',
 'DEPORTE JUGUETES',
 'DULE',
 'HOME E-MACHINE',
 'VERDULEROS',
 'HORA DEL TE',
 '._.DS_Store',
 'PLATOS PREPARADOS',
 'ABSTEMIOS IMPULSIVOS',
 '.DS_Store',
 'FERRE AUTO  JARDIN',
 'TUTIFRUTI']

In [3]:
if '._.DS_Store' in cat_folders:
    cat_folders.remove('._.DS_Store')

In [4]:
if '.DS_Store' in cat_folders:
    cat_folders.remove('.DS_Store')

In [5]:

for folder in cat_folders:
    file = os.path.join(root, folder, 'todo.csv')
    data = pd.read_csv(file)
    ids = list(set(data.barcode.to_list()))
    ids_array = np.array(ids)
    val_set = np.random.choice(ids_array, round(0.2*len(ids)))
    ids_set = set(ids)
    val = set(val_set)
    train = ids_set - val
    traincodes = list(train)
    valcodes = list(val)
    pd_train = data
    pd_val = data
    for code in valcodes:
        pd_train = pd_train[pd_train.barcode != code]
    for code in traincodes:
        pd_val = pd_val[pd_val.barcode != code]
    train_file = os.path.join(root, folder, 'train.csv')
    val_file = os.path.join(root, folder, 'val.csv')
    pd_train.to_csv(train_file, index=False)
    pd_val.to_csv(val_file, index=False)
    change = {0:1, 1:0}
    pd_train['is_promo'] = pd_train.true_classification.map(change)
    pd_val['is_promo'] = pd_val.true_classification.map(change)
    pd_train['date'] = pd.to_datetime(pd_train['date'])
    pd_val['date'] = pd.to_datetime(pd_val['date'])
    barcodes_train = list(set(pd_train.barcode.to_list()))
    pos2barcode_train = pd.DataFrame(barcodes_train, columns=['barcode'])
    barcodes_val = list(set(pd_val.barcode.to_list()))
    pos2barcode_val = pd.DataFrame(barcodes_val, columns=['barcode'])
    for idx in range(len(barcodes_train)):
        barcode = pos2barcode_train.barcode[idx]
        arrow_data = pd_train[pd_train.barcode == barcode].sort_values(by='date',ascending=False)
        sample = torch.tensor(arrow_data.price.to_list())
        label = torch.tensor(arrow_data.is_promo.to_list())
        sample_path =  os.path.join(root, folder, 'train')
        if not os.path.exists(sample_path):
            os.makedirs(sample_path)
        input_path = os.path.join(sample_path, 'inputs')
        if not os.path.exists(input_path):
            os.makedirs(input_path)
        labels_path = os.path.join(sample_path, 'labels')
        if not os.path.exists(labels_path):
            os.makedirs(labels_path)
        sample_path =  os.path.join(input_path, str(barcode) + '.pt')
        label_path =  os.path.join(labels_path, str(barcode) + '.pt')
        torch.save(sample, sample_path)
        torch.save(label, label_path)
    for idx in range(len(barcodes_val)):
        barcode = pos2barcode_val.barcode[idx]
        arrow_data = pd_val[pd_val.barcode == barcode].sort_values(by='date',ascending=False)
        sample = torch.tensor(arrow_data.price.to_list())
        label = torch.tensor(arrow_data.is_promo.to_list())
        sample_path =  os.path.join(root, folder, 'val')
        if not os.path.exists(sample_path):
            os.makedirs(sample_path)
        input_path = os.path.join(sample_path, 'inputs')
        if not os.path.exists(input_path):
            os.makedirs(input_path)
        labels_path = os.path.join(sample_path, 'labels')
        if not os.path.exists(labels_path):
            os.makedirs(labels_path)
        sample_path =  os.path.join(input_path, str(barcode) + '.pt')
        label_path =  os.path.join(labels_path, str(barcode) + '.pt')
        torch.save(sample, sample_path)
        torch.save(label, label_path)
